In [ ]:
!pip install s3fs

In [1]:
import configparser
import psycopg2
config = configparser.ConfigParser()
config.read('dwh.cfg')

KEY=config.get('AWS','key')
SECRET= config.get('AWS','secret')

HOST= config.get("CLUSTER","HOST")
DB_NAME= config.get("CLUSTER","DB_NAME")
DB_USER= config.get("CLUSTER","DB_USER")
DB_PASSWORD= config.get("CLUSTER","DB_PASSWORD")
DB_PORT = config.get("CLUSTER","DB_PORT")


IAM_ROLE             = config.get('IAM_ROLE', 'ARN')
t1        = config.get('S3', 't1')
testcrm1        = config.get('S3', 'testcrm1')
testdevice1        = config.get('S3', 'testdevice1')
testrev01        = config.get('S3', 'testrev01')
REGION          = config.get('CLUSTER', 'DB_REGION')
S3_REGION = config.get('S3', 'S3_REGION')
print(*config['CLUSTER'].values())

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()


redshift-cluster-1.cftmnj5okklz.us-west-2.redshift.amazonaws.com dev awsuser Survesh0101 5439 us-west-2


In [2]:
import boto3
#import s3fs
import pandas as pd
from time import time

In [3]:


s3 = boto3.resource('s3',
                    region_name="us-west-2",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                   )
sampleDbBucket =  s3.Bucket("surveshcap")


In [4]:
sampleDbBucket.name

'surveshcap'

In [5]:
keys_json  = []
keys_csv  = []
for i in sampleDbBucket.objects.iterator():
    _,etn = (i.key.split("."))
    if(etn=='json'):
        keys_json.append(i.key)
    else:
        keys_csv.append(i.key)
keys_json,keys_csv

(['t1.json', 'testrev01.json'], ['testcrm1.csv', 'testdevice1.csv'])

In [6]:
%run create_tables.py

Running drop and create table process
Tables is successfully dropped .
Tables is successfully created .
Drop and Create Table is DONE


In [7]:
# for 1st json file
link='s3://'+ sampleDbBucket.name + '/' + keys_json[0]
print(link)

s3://surveshcap/t1.json


In [8]:
config = configparser.ConfigParser()
config.read('dwh.cfg')
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [9]:
S3_REGION

'us-west-1'

In [18]:
query = ("""copy rev_stage 
                          from {}
                          iam_role '{}'
                          json '{}'
                          region '{}'
                       """).format(t1, IAM_ROLE,'auto',S3_REGION)


print(query)

copy rev_stage 
                          from 's3://surveshcap/t1.json'
                          iam_role 'arn:aws:iam::559792065225:role/airflow_redshift_user'
                          json 'auto'
                          region 'us-west-1'
                       


In [20]:
cur.execute(query)
conn.commit()